# GA4 API Tool

In [ ]:
pip install pandas numpy matplotlib google-analytics-data oauth2client

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import FileLink
from oauth2client.service_account import ServiceAccountCredentials
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange, Dimension, Metric, RunReportRequest

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'client_secrets.json'
property_id = 0
web_name=""

client = BetaAnalyticsDataClient()

In [ ]:
def make_a_request(start_date, end_date, dimension, metric):
    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=[Dimension(name=dimension)],
        metrics=[Metric(name=metric)],
        date_ranges=[DateRange(start_date=start_date, end_date=end_date)],
    )
    response = client.run_report(request)
    return response

## Los navegadores desde donde nos llegan mas visitas

In [ ]:
response = make_a_request("2020-03-31", "today", "browser", "activeUsers")
output = []
for row in response.rows:
    output.append(
        {
            'Browser': row.dimension_values[0].value,
            'Users': row.metric_values[0].value
        }
    )

browser_df = pd.DataFrame.from_dict(output)

fig, ax = plt.subplots()
ax.pie(browser_df['Users'], radius=3, autopct='%1.1f%%', center=(4, 4), frame=True)
ax.legend(browser_df['Browser'], loc='center left', bbox_to_anchor=(1, 0.5))
plt.axis('off')
plt.show()

## Los paises desde donde nos llegan mas visitas

In [ ]:
response = make_a_request("2020-03-31", "today", "country", "activeUsers")
output = []

for row in response.rows:
    output.append(
        {
            'Country': row.dimension_values[0].value,
            'Users': row.metric_values[0].value
        }
    )

country_df = pd.DataFrame.from_dict(output)

fig, ax = plt.subplots()
ax.pie(country_df['Users'], radius=3, autopct='%1.1f%%', center=(4, 4), frame=True)
ax.legend(country_df['Country'][0:5], loc='center left', bbox_to_anchor=(1, 0.5))
plt.axis('off')
plt.show()

## Trafico por medio

In [ ]:
response = make_a_request("2020-03-31", "today", "medium", "activeUsers")
output = []

for row in response.rows:
    output.append(
        {
            'Medium': row.dimension_values[0].value,
            'Users': row.metric_values[0].value
        }
    )

medium_df = pd.DataFrame.from_dict(output)

fig, ax = plt.subplots()
plt.margins(0.1)
x_pos = np.arange(len(medium_df['Medium']))

bar = ax.bar(x_pos, medium_df['Users'], align='center')
ax.bar_label(bar, labels=medium_df['Users'], padding = 3) 
ax.set_xticks(x_pos, labels=medium_df['Medium'])
ax.set_ylabel('Usuarios')
ax.set_xlabel('Medio')
ax.set_title('Trafico / Medio')
plt.show()

## Trafico de mobile, desktop y tablet

In [ ]:
response = make_a_request("2020-03-31", "today", "deviceCategory", "activeUsers")
output = []

for row in response.rows:
    output.append(
        {
            'Device': row.dimension_values[0].value,
            'Users': row.metric_values[0].value
        }
    )

device_df = pd.DataFrame.from_dict(output)

fig, ax = plt.subplots()
ax.pie(device_df['Users'], radius=3, autopct='%1.1f%%', center=(4, 4), frame=True)
ax.legend(device_df['Device'], loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_title('Dispositivo / Medio')
plt.axis('off')
plt.show()

## CSV con las urls y el trafico

In [ ]:
response = make_a_request("2020-03-31", "today", "fullPageUrl", "activeUsers")
output = []

for row in response.rows:
    output.append(
        {
            'Page URL': row.dimension_values[0].value,
            'Users': row.metric_values[0].value
        }
    )

urls_df = pd.DataFrame.from_dict(output)
file_name = f'Traffic-urls-{web_name}.csv'
urls_df.to_csv(file_name, index=False)

print('Descarga el Informe:')
display(FileLink(file_name))
